In [16]:
import QUANTAXIS as qa
from QIFIAccount import QIFI_Account,QA_Position
from QIFIAccount.QASimAccount import QIFI_SIM_Account
import pandas as pd

In [17]:
my=QIFI_SIM_Account(username='myacc4',password='myacc4')
my.initial()

In [12]:
df=pd.read_excel("E:\\leo\\投资\\2021\\股票持仓分析20210509-快照20210520.xlsx",sheet_name="持仓明细详情",header=6)
df=df[df['pos_num']>0]
code_cn_list=list(x.split(".")[0] for x in df.code[df['currency']=='CNY'].unique().tolist())
code_cn_list

['512800',
 '600036',
 '601166',
 '601225',
 '601318',
 '000961',
 '160631',
 '510880',
 '002027',
 '000333',
 '002353',
 '502020',
 '160322',
 '161121',
 '600516',
 '601668',
 '601939',
 '510050',
 '510900',
 '159920',
 '600985']

In [18]:
code_hk_list=list(x.split(".")[0] for x in df.code[df['currency']=='HK'].unique().tolist())
df_cn=df[df['currency']=='CNY'].assign(code=df['code'].apply(lambda x:x.split(".")[0]))

def QA_fetch_fund_name(code, collections=qa.DATABASE.fund_list):
    """
    获取股票名称
    """
    if isinstance(code, str):
        try:
            res = collections.find_one({'code': code})
            return res['name']
        except Exception as e:
            if (res is None):
                return qa.QA_fetch_etf_name(code)
            return code
def cn_mkt():
    real_price=qa.QA_fetch_get_stock_realtime("tdx",code=code_cn_list).reset_index(drop=False)
    real_price=real_price.assign(r=((real_price['price']/real_price['last_close']-1)*100).round(2))
    real_price=real_price.assign(last_close=real_price.apply(lambda x:x['last_close'] if x['code'][0] in ['0','3','6'] else x['last_close']/10,axis=1),
                             price=real_price.apply(lambda x:x['price'] if x['code'][0] in ['0','3','6'] else x['price']/10,axis=1))
    real_price=real_price.assign(mkt=real_price.apply(lambda x:'stock' if x['code'][0] in ['0','3','6'] else 'fund',axis=1))
    
    real_price=real_price.assign(name=real_price['code'].apply(lambda x:qa.QA_fetch_stock_name(x) if x[0] in ['0','3','6'] else QA_fetch_fund_name(x)),pos=real_price['code'].apply(lambda x:df_cn[df_cn['code']==x]['pos_num'].sum()))
    real_price=real_price.assign(mv=real_price['price']*real_price['pos'])

    real=real_price[['code','name','last_close','price','r','pos','mv','mkt']].assign(pofit=(real_price['pos']*real_price['last_close']*real_price['r']/100).round(2),w=(real_price['mv']/real_price['mv'].sum()*100).round(2)).sort_values('r',ascending=False)
    print('total:%s %s'% (real['mv'].sum().round(2),real['pofit'].sum().round(2)))
    return real

In [14]:
cn_real=cn_mkt()

total:1682618.01 13254.62

In [22]:
code_hk_list=[i.rjust(5,'0') if len(i)==4  else i for i in code_hk_list]

In [15]:
cn_stock=cn_real[cn_real['mkt']=='stock']
cn_stock

,code,name,last_close,price,r,pos,mv,mkt,pofit,w
1,600036,招商银行,56.28,57.53,2.22,3300.0,189849.00,stock,4123.07,11.28
8,002027,分众传媒,9.76,9.95,1.95,12700.0,126365.00,stock,2417.06,7.51
2,601166,兴业银行,22.10,22.39,1.31,32200.0,720958.00,stock,9322.22,42.85
16,601939,建设银行,6.77,6.81,0.59,3600.0,24516.00,stock,143.79,1.46
4,601318,中国平安,70.68,70.85,0.24,900.0,63765.00,stock,152.67,3.79
15,601668,中国建筑,4.88,4.89,0.20,2000.0,9780.00,stock,19.52,0.58
10,002353,杰瑞股份,40.67,40.74,0.17,300.0,12222.00,stock,20.74,0.73
5,000961,中南建设,6.79,6.79,0.00,13500.0,91665.00,stock,0.00,5.45
9,000333,美的集团,77.93,77.73,-0.26,374.0,29071.02,stock,-75.78,1.73
14,600516,方大炭素,8.33,8.29,-0.48,1043.0,8646.47,stock,-41.70,0.51


In [23]:
code_hk_list

['01088', '01516', '01918', '02007', '200625', '00700']

In [9]:
def get_pos(stock):
    return QA_Position(
        code=stock['code'],
        account_cookie='myacc4',
        market_type=qa.MARKET_TYPE.STOCK_CN,
        exchange_id=qa.EXCHANGE_ID.SZSE if stock['code'][0] in (0,3) else qa.EXCHANGE_ID.SSE,
        volume_long_his=stock['pos'],
        volume_long_today=0,
        open_price_long=stock['price'],
        name=stock['name']
    )
for pos in cn_stock.apply(lambda x:get_pos(x),axis=1):
    my.add_position(pos)

settle